# Regras de Associação com o Algoritmo Apriori

## **Objetivo**
O objetivo deste estudo é identificar padrões e associações relevantes nos dados, utilizando o algoritmo **Apriori**.

Esta técnica é amplamente aplicada em mineração de dados para descobrir regras de associação, que podem ser utilizadas para entender relações entre variáveis ou itens no conjunto de dados.

## **Estrutura do Processo**
1. **Definição do objetivo de negócio:**
   - Identificar relações frequentes e associações entre itens nos dados.

2. **Preparação dos dados:**
   - Realizar a transformação dos dados para o formato adequado para o algoritmo Apriori.
   - Limpeza e formatação dos dados (se necessário).

3. **Aplicação do algoritmo Apriori:**
   - Configurar parâmetros como suporte, confiança e lift para gerar regras de associação relevantes.
   - Avaliar as regras descobertas com base em métricas de qualidade.

4. **Documentação dos resultados:**
   - Apresentar os resultados intermediários (ex.: conjuntos frequentes) e finais (regras de associação).
   - Discutir a relevância e aplicação prática das associações descobertas.




In [ ]:
# Imports
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [6]:
# Carrega os ficheiros
from google.colab import files

# Faz o upload do ficheiro de dados de habitação da Califórnia
uploaded = files.upload()

for fn in uploaded.keys():
    print('O utilizador carregou o ficheiro "{name}" com tamanho {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

Saving housing_sl.csv to housing_sl.csv
O utilizador carregou o ficheiro "housing_sl.csv" com tamanho 1968649 bytes


In [7]:
# 0. Lê o ficheiro
cali_file = pd.read_csv('housing_sl.csv')
housing_df = pd.DataFrame(cali_file)

housing_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,house_value_range,Proximity_Level_1O,Proximity_Level_B,Proximity_Level_IN,Proximity_Level_IS,Proximity_Level_NO
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,350000,False,True,False,False,False
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,350000,False,True,False,False,False
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,350000,False,True,False,False,False
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,350000,False,True,False,False,False
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,350000,False,True,False,False,False


### Preparação dos Dados

Os dados foram previamente limpos, com:
- Remoção de outliers,
- Substituição de valores nulos,
- Aplicação de one-hot encoding para variáveis categóricas como `ocean_proximity`.

### Substituição dos Intervalos de Preço por Valores Numéricos Médios

Primeiro, foi feito o agrupamento dos preços das casas (`median_house_value`) em intervalos. Esses intervalos foram mapeados para valores médios para facilitar a previsão.

```python
house_value_mapping = {
    '0-15000': 7500,
    '15000-30000': 22500,
    '30000-50000': 40000,
    '50000-100000': 75000,
    '100000-200000': 150000,
    '200000-500000': 350000
}
cali_df['house_value_range'] = cali_df['house_value_range'].map(house_value_mapping)
```
Após essa transformação, a variável `house_value_range` tornou-se numérica, representando o intervalo médio do preço das casas.

### Tabela Pivot

Para usar o algoritmo Apriori, é necessário transformar os dados em uma tabela pivot.

Essa tabela será utilizada para identificar associações entre os itens ou transações.

Na tabela pivot:

 - Se o produto estiver presente na transação, a célula correspondente será marcada como “True”.
 - Se o produto não estiver presente na transação, a célula correspondente será marcada como “False”.

In [19]:
# Agrupar os dados por 'households' e 'house_value_range'
grouped_data = housing_df.groupby(['households', 'house_value_range'], as_index=False).agg({'total_rooms': 'sum'})

# Criar a tabela pivot para Apriori
pivot_table = pd.pivot_table(
    data=grouped_data,
    index='households',                # Cada agregado familiar
    columns='house_value_range',      # Faixas de valor das casas
    values='total_rooms',             # Quantidade total de quartos
    fill_value=0                      # Preencher valores ausentes com 0
).map(lambda x: True if x > 0 else False)  # Transformar em booleano

# Visualizar as últimas linhas da tabela pivot
pivot_table.tail()

house_value_range,7500,22500,40000,75000,150000,350000
households,,,,,,
4855.0,False,False,False,False,False,True
5050.0,False,False,False,False,False,True
5189.0,False,False,False,False,False,True
5358.0,False,False,False,False,True,False
6082.0,False,False,False,False,True,False


## Resultado

### Colunas de Faixa de Preços:

As colunas representam as faixas de preço mapeadas:

 - 7500 para 0-15000
 - 22500 para 15000-30000
 - 40000 para 30000-50000
 - 75000 para 50000-100000
 - 150000 para 100000-200000
 - 350000 para 200000-500000

Está correto e segue a estrutura esperada.

### Linhas de households:

Cada linha representa um agregado familiar `(households)` e indica se há presença de quartos para cada faixa de preço.

Os valores `True/False` refletem corretamente a presença `(ou ausência)` de quartos em uma faixa de preço específica.
Consistência:

Para os valores de households **4855.0**, **5050.0**, e **5189.0**, a faixa **350000** é marcada como `True`, o que faz sentido para agregados com valores de casa mais altos.

Para **5358.0** e **6082.0**, a faixa **150000** é marcada como `True`, o que é consistente com faixas intermediárias.

## Learning Rules (Association Rule Learning)

In [27]:
# Gerar os itemsets frequentes com o algoritmo Apriori
min_support = 0.01
freq_itemsets = apriori(pivot_table, min_support=min_support, use_colnames=True)

# Contar o número de itemsets
num_itemsets = len(freq_itemsets)
print(f'Number of itemsets: {num_itemsets}')
freq_itemsets.head()

# Gerar as regras de associação (sem 'num_itemsets')
rules = association_rules(freq_itemsets, metric="support", min_threshold=min_support, num_itemsets=num_itemsets)

# Classificar as regras por suporte e exibir as 10 principais
rules.sort_values('support', ascending=False).head(10)

Number of itemsets: 15


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
11,(350000),(150000),0.817066,0.810931,0.644172,0.788396,0.972210,1.0,-0.018413,0.893502,-0.135138,0.654762,-0.119192,0.791378
10,(150000),(350000),0.810931,0.817066,0.644172,0.794360,0.972210,1.0,-0.018413,0.889584,-0.131328,0.654762,-0.124121,0.791378
6,(75000),(150000),0.510318,0.810931,0.476297,0.933333,1.150940,1.0,0.062464,2.836029,0.267816,0.563696,0.647394,0.760339
7,(150000),(75000),0.810931,0.510318,0.476297,0.587345,1.150940,1.0,0.062464,1.186663,0.693637,0.563696,0.157301,0.760339
8,(75000),(350000),0.510318,0.817066,0.474066,0.928962,1.136948,1.0,0.057102,2.575143,0.245980,0.555556,0.611672,0.754583
9,(350000),(75000),0.817066,0.510318,0.474066,0.580205,1.136948,1.0,0.057102,1.166478,0.658447,0.555556,0.142719,0.754583
35,(150000),"(75000, 350000)",0.810931,0.474066,0.455661,0.561898,1.185275,1.0,0.071226,1.200484,0.826757,0.549428,0.167003,0.761537
34,(350000),"(75000, 150000)",0.817066,0.476297,0.455661,0.557679,1.170865,1.0,0.066495,1.183990,0.797724,0.543941,0.155398,0.757177
33,(75000),"(350000, 150000)",0.510318,0.644172,0.455661,0.892896,1.386115,1.0,0.126929,3.322274,0.568857,0.652035,0.699001,0.800128
32,"(350000, 150000)",(75000),0.644172,0.510318,0.455661,0.707359,1.386115,1.0,0.126929,1.673322,0.782847,0.652035,0.402386,0.800128


In [28]:
# Lista as 10 rules com maior confiança
rules.sort_values('confidence', ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
12,"(40000, 75000)",(150000),0.089794,0.810931,0.089236,0.993789,1.225491,1.0,0.016419,30.440045,0.202152,0.109966,0.967149,0.551915
36,"(40000, 350000, 75000)",(150000),0.088678,0.810931,0.088120,0.993711,1.225394,1.0,0.016209,30.061907,0.201834,0.108591,0.966735,0.551188
18,"(40000, 75000)",(350000),0.089794,0.817066,0.088678,0.987578,1.208687,1.0,0.015311,14.726157,0.189689,0.108384,0.932094,0.548055
38,"(40000, 75000, 150000)",(350000),0.089236,0.817066,0.088120,0.987500,1.208592,1.0,0.015209,14.634690,0.189501,0.107703,0.931669,0.547675
4,(40000),(350000),0.094255,0.817066,0.092582,0.982249,1.202165,1.0,0.015569,10.305261,0.185668,0.113079,0.902962,0.547780
25,"(40000, 150000)",(350000),0.092025,0.817066,0.090351,0.981818,1.201638,1.0,0.015161,10.061350,0.184810,0.110354,0.900610,0.546199
41,"(40000, 75000)","(350000, 150000)",0.089794,0.644172,0.088120,0.981366,1.523455,1.0,0.030278,19.096115,0.377494,0.136442,0.947633,0.559081
2,(40000),(150000),0.094255,0.810931,0.092025,0.976331,1.203963,1.0,0.015590,7.988148,0.187039,0.113169,0.874815,0.544906
24,"(40000, 350000)",(150000),0.092582,0.810931,0.090351,0.975904,1.203435,1.0,0.015274,7.846347,0.186293,0.111111,0.872552,0.543660
37,"(40000, 350000, 150000)",(75000),0.090351,0.510318,0.088120,0.975309,1.911179,1.0,0.042013,19.832125,0.524117,0.171926,0.949577,0.573993


In [30]:
# Cria um DataFrame com 'house_value_range' e 'households' distintos
products_list = housing_df[['house_value_range', 'households']].drop_duplicates()

# Visualiza as primeiras 10 linhas
products_list.head(10)


,house_value_range,households
0,350000,126.0
1,350000,1138.0
2,350000,177.0
3,350000,219.0
4,350000,259.0
5,350000,193.0
6,350000,514.0
7,350000,647.0
8,350000,595.0
9,350000,714.0


In [33]:
# Função para obter a descrição do valor das faixas de preço
def get_house_value_description(value_range):
    try:
        return str(value_range)  # Garantir que o valor seja tratado como string
    except IndexError:
        return "Description not found"

# Função para obter as regras relacionadas ao valor de faixa de preço
def get_rules_for_price_range(value_range):

    # Filtrar as regras onde o 'house_value_range' está nos antecedentes
    filtered_rules = rules[rules['antecedents'].apply(lambda x: value_range in list(x))]

    # Preparar os resultados
    results = []
    for index, row in filtered_rules.iterrows():
        rule_info = {}

        # Obter as descrições dos antecedentes e consequentes
        antecedents_descriptions = [get_house_value_description(code) for code in list(row['antecedents'])]
        consequents_descriptions = [get_house_value_description(code) for code in list(row['consequents'])]

        rule_info['antecedents'] = ", ".join(antecedents_descriptions)
        rule_info['consequents'] = ", ".join(consequents_descriptions)
        rule_info['support'] = row['support']
        rule_info['confidence'] = row['confidence']
        results.append(rule_info)

    # Converter os resultados para DataFrame e classificar por confiança
    result_df = pd.DataFrame(results)
    result_df = result_df.sort_values('confidence', ascending=False).head(10)

    return result_df

# Exemplo de como usar a função com um valor específico de 'house_value_range'
product_rules = get_rules_for_price_range(350000)  # Por exemplo, 350000 (faixa de preço mais alta)
product_rules


,antecedents,consequents,support,confidence
12,"40000, 350000, 75000",150000,0.088120,0.993711
6,"40000, 350000",150000,0.090351,0.975904
13,"40000, 350000, 150000",75000,0.088120,0.975309
9,"75000, 350000",150000,0.455661,0.961176
3,"40000, 350000",75000,0.088678,0.957831
15,"40000, 350000","75000, 150000",0.088120,0.951807
2,350000,150000,0.644172,0.788396
10,"350000, 150000",75000,0.455661,0.707359
1,350000,75000,0.474066,0.580205
11,350000,"75000, 150000",0.455661,0.557679


# Conclusão

Neste notebook, aplicamos o algoritmo **Apriori** para identificar padrões e regras de associação no conjunto de dados sobre imóveis na Califórnia. O objetivo principal foi analisar as faixas de preço das casas e a quantidade de quartos presentes em diferentes **households**, para identificar associações entre essas variáveis.

As etapas seguidas foram:
1. **Preparação dos dados:** Realizamos o agrupamento dos dados e transformamos os valores de quantidade de quartos numa tabela pivot booleana, pronta para ser utilizada pelo algoritmo Apriori.

2. **Aplicação do Apriori:** Geramos os **itemsets frequentes** com o parâmetro de **suporte mínimo** ajustado para 0.01, e, a partir disso, calculamos as **regras de associação**.

3. **Análise das regras:** As regras foram filtradas com base no **suporte** e **confiança**, e as descrições dos **antecedentes** e **consequentes** foram exibidas.

No final, o notebook gerou uma análise das associações mais significativas, com insights sobre as relações entre os valores das casas e as características dos agregados familiares.

Esses resultados podem ser utilizados para entender melhor como as faixas de preço das casas estão distribuídas em diferentes regiões e como os itens estão associados.

Essa análise pode ser útil para decisões de mercado, como segmentação de imóveis e estratégias de precificação baseadas em dados históricos.
